In [5]:
import requests as req
from bs4 import BeautifulSoup as bs

In [49]:
from urllib.parse import urlencode

# URL Example
# https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy=2018&mm=9&obs=9
query_dict = dict({
    "stn": "108", # 지역번호
    "yy": "2018", # 연도
    "mm": "09", # 월
    "obs": "9" # 데이터 구분 1 - 기온/강수량, 9 - 날씨
})
query = urlencode(query_dict)

base_url = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do"
res = req.get("{}?{}".format(base_url, query))
weather_soup = bs(res.content, "html.parser")
weather_soup.select("tbody > tr > td")

[<td>무인관측요소 + 날씨(기상현상), 구름, 적설</td>,
 <td>기온, 강수량, 바람, 습도, 일조시간</td>,
 <td>기존 관측지점에 (구)를 붙여 표시</td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span>1일</span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span> </span>
 </td>,
 <td>
 <span>2일</span>
 </td>,
 <td>
 <span>3일</span>
 </td>,
 <td>
 <span>4일</span>
 </td>,
 <td>
 <span>5일</span>
 </td>,
 <td>
 <span>6일</span>
 </td>,
 <td>
 <span>7일</span>
 </td>,
 <td>
 <span>8일</span>
 </td>,
 <td>
 <span>비</span>
 <span></span>
 <span><br/></span>
 <span>박무</span>
 </td>,
 <td>
 <span>박무</span>
 </td>,
 <td>
 <span>비</span>
 <span></span>
 <span><br/></span>
 <span>박무</span>
 </td>,
 <td>
 <span>비</span>
 </td>,
 <td>
 <span>9일</span>
 </td>,
 <td>
 <span>10일</span>
 </td>,
 <t

In [48]:
from datetime import datetime, timedelta

startDate = datetime(2018,5,1)
endDate = datetime(2019,5,1)
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
dateIndexArr = [dt for dt in
                   datetime_range(startDate, endDate, timedelta(days = 1))]

dates = list(set([dt.strftime("%Y%m") for dt in dateIndexArr]))
dates.sort()
dates

for date in dates:
    yy = date[0:4]
    mm = date[-2:]
    weather_query_dict = dict({
        "stn": "108", # 지역번호
        "yy": yy, # 연도
        "mm": mm, # 월
        "obs": "9" # 데이터 구분 1 - 기온/강수량, 9 - 날씨
    })
    weather_query = urlencode(query_dict)

    base_url = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do"
    res = req.get("{}?{}".format(base_url, weather_query))
    
    weather_soup = bs(res.content, "html.parser")
    weather_soup.select("tbody > tr > td")

2018
05
2018
06
2018
07
2018
08
2018
09
2018
10
2018
11
2018
12
2019
01
2019
02
2019
03
2019
04
